In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib as mlt
import re
import math

In [7]:
#get datasets
imdb = pd.read_csv("../datasets/imdb.csv")
filmmv = pd.read_csv("../datasets/filmtv_movies.csv")
filmmv

,filmtv_id,title,year,genre,duration,country,directors,actors,avg_vote,critics_vote,public_vote,total_votes,description,notes,humor,rhythm,effort,tension,erotism
0,2,Bugs Bunny's Third Movie: 1001 Rabbit Tales,1982,Animation,76,United States,"David Detiege, Art Davis, Bill Perez",NaN,7.7,8.00,7.0,22,"With two protruding front teeth, a slightly sl...","These are many small independent stories, whic...",3,3,0,0,0
1,3,18 anni tra una settimana,1991,Drama,98,Italy,Luigi Perelli,"Kim Rossi Stuart, Simona Cavallari, Ennio Fant...",6.5,6.00,7.0,4,"Samantha, not yet eighteen, leaves the comfort...","Luigi Perelli, the director of the ""Piovra"", o...",0,2,0,2,0
2,17,Ride a Wild Pony,1976,Romantic,91,United States,Don Chaffey,"Michael Craig, John Meillon, Eva Griffith, Gra...",5.7,6.00,5.0,10,"In the Australia of the pioneers, a boy and a ...","""Ecological"" story with a happy ending, not wi...",1,2,1,0,0
3,18,Diner,1982,Comedy,95,United States,Barry Levinson,"Mickey Rourke, Steve Guttenberg, Ellen Barkin,...",7.0,8.00,6.0,18,Five boys from Baltimore have a habit of meeti...,A cast of will be famous for Levinson's direct...,2,2,0,1,2
4,20,A che servono questi quattrini?,1942,Comedy,85,Italy,Esodo Pratelli,"Eduardo De Filippo, Peppino De Filippo, Clelia...",5.9,5.33,7.0,15,"With a stratagem, the penniless and somewhat p...",Taken from the play by Armando Curcio that the...,3,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41394,232817,Gold Digger Killer,2021,Thriller,87,"Canada, United States",Robin Hays,"Julie Benz, Roan Curtis, Georgia Bradner, Eli ...",4.0,NaN,4.0,3,Celeste is an attractive waitress in the forti...,Freely taken from a true story.,0,0,0,0,0
41395,232893,Addio al nubilato 2,2023,Comedy,90,Italy,Francesco Apolloni,"Laura Chiatti, Chiara Francini, Antonia Liskov...",2.7,NaN,3.0,3,"When Eleonora is downloaded to the altar, her ...",Bachelorette fare sequel (2021).,0,0,0,0,0
41396,232915,Konferensen,2023,Horror,100,Sweden,Patrik Eklund,"Katia Winter, Eva Melander, Lola Zackow, Adam ...",6.0,NaN,6.0,6,A team building conference organized for a gro...,NaN,0,0,0,0,0
41397,232919,Ballelina,2023,Thriller,92,South Korea,Chung-Hyun Lee,"Jeon Jong-seo, Park Yu-rim, Ji-hun Kim",5.8,NaN,6.0,5,Once Ok-Ju worked as a bodyguard and was one o...,NaN,0,0,0,0,0


In [3]:
#tokenize sentences by words
vocabluary = []
sentences = []
for i in range(len(filmmv)):
    try:
        vocabluary.extend(list(map(lambda x:x.lower(),filmmv["description"][i].split())))
        sentences.append(filmmv["description"][i])
    except:
        continue
vocabluary = list(set(vocabluary))

dict = {}
maxx = len(max(sentences,key=len).split())
for i in range(len(vocabluary)):
    dict.update({vocabluary[i]:i+1})
index = 0
tokenize_sentences = np.zeros([maxx])
for i in range(100):
    temp = sentences[i].split()
    temp_tokenize_sentences = np.zeros([maxx])
    for word in range(len(temp)):
        temp_val = dict[temp[word].lower()]
        np.put(temp_tokenize_sentences,(0,word), temp_val)
    tokenize_sentences = np.vstack((tokenize_sentences, temp_tokenize_sentences))


In [4]:
#100 tokenize sentences 
tokenize_sentences

array([[     0.,      0.,      0., ...,      0.,      0.,      0.],
       [ 30441.,  78416.,  49201., ...,      0.,      0.,      0.],
       [ 99299.,  46137.,  48594., ...,      0.,      0.,      0.],
       ...,
       [ 25046.,   7381., 128952., ...,      0.,      0.,      0.],
       [ 82992.,  49017.,  84438., ...,      0.,      0.,      0.],
       [117236., 120976.,  16349., ...,      0.,      0.,      0.]])

In [5]:
def softmax(input_arr):
    all_sum = sum(input_arr)
    for i in range(len(input_arr)):
        if input_arr[i] == 0:
            input_arr[i] == 0
        else:
            temp = input_arr[i]
            input_arr[i] = temp/all_sum
    
    return input_arr

In [6]:
print(softmax(tokenize_sentences[2]))

[0.01961951 0.00911575 0.00960121 0.02112191 0.01183288 0.02306392
 0.00789372 0.00757384 0.01848876 0.01590579 0.02526576 0.01136402
 0.01830797 0.01231714 0.00769377 0.01573982 0.01177578 0.00506477
 0.01462449 0.00812212 0.02543706 0.02542955 0.01590579 0.00841889
 0.01787132 0.0016575  0.00968478 0.0080259  0.00628542 0.01241119
 0.02165972 0.02078444 0.00757384 0.00812212 0.00796307 0.00757384
 0.01345363 0.00808241 0.00602876 0.00812212 0.00607144 0.01978172
 0.01241119 0.02165972 0.02078444 0.00757384 0.02306392 0.00982882
 0.01189827 0.0015599  0.00240415 0.02306392 0.01061657 0.00757384
 0.02306392 0.01572185 0.01590579 0.01852669 0.01006868 0.0015599
 0.00068837 0.01195735 0.00899701 0.01177578 0.02165972 0.01607433
 0.00757384 0.02306392 0.01888846 0.02207523 0.01679629 0.02104762
 0.02165972 0.01961951 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.      